In [ ]:
import zmq
import telemetry_pb2

context = zmq.Context()
poller = zmq.Poller()

ports = [5556, 5557, 5558, 5559, 5560]  
sockets = []

# Set up a socket for each port
for port in ports:
    sock = context.socket(zmq.PULL)
    sock.connect(f"tcp://127.0.0.1:{port}")
    poller.register(sock, zmq.POLLIN)
    sockets.append((sock, port))
#making sure we are connected
print(f"Listening on ports {ports}...")
while True:
    try:
        events = dict(poller.poll())

        for sock, port in sockets:
            if sock in events:
                msg_bytes = sock.recv()
                msg = telemetry_pb2.TelemetryData()
                msg.ParseFromString(msg_bytes)
                print(f" Received from port {port}: timestamp={msg.unix_timestamp}, lat={msg.latitude}")

    except KeyboardInterrupt:
        print("\n Receiver stopped.")
        break